## <center> Lab Assignment 4

Add libraries


### Problem 1

Let $A=\begin{bmatrix} -6 & 28 & 21 \\ 4 & -15 & -12  \\ -8 & a & 25   \end{bmatrix} $.

**a.** For each value of $a$ in the set $\{32, 31.9, 31.8,32.1, 32.2\}$, compute characteristic polynomial of $A$ and the eigenvalues. In each case, draw the graph of characteristic polynomial for $0 \leq \lambda \leq 3$ in the same coordinate system. Describe how the graphs reveal the changes in the eigenvalues as $a$ changes.

**Answer:**

**b.** Let $a=32$. Do $A$ and $A^T$ have same eigenvalues? Do they have the same eigenvectors?

**Answer:**

### Problem 2

In this problem we will compare speed of calculating steady-state vector for a large stochastic matrix using two different methods:
1. Finding basis for Null Space of $P-I$
2. Finding image of $P^k\bf{v}$ for some large $k$.

The code below will generate a random 50 by 50 stochastic matrix for you. (Note: be careful, if you re-run this command in the middle of your editing, it will change the matrix and you will have to rerun the whole problem again)

In [ ]:
matrix = np.random.rand(50,50)
S=matrix/matrix.sum(axis=0)[:,None]

**a.** For generated matrix $S$ find  steady-state vector by finding basis for null_space of $S-I$.

**b.** For generated matrix $S$ find steady-state vector by taking the first column $\bf{v}$ of $S$, and calculating $S^{200}\bf{v}$. Normalize the vector. By looking at the results, confirm you got the same answer. (Note: the vectors may differ by sign. That is ok)

**c.** Rerun both methods again, this time timing them using `%%timeit -n 100 -r 40 `. Which method is faster? Which method do you prefer?

**Answer:**

### Problem 3

In this problem we will use eigenvectors to look at something we will discuss in more details in the future. We start with uploading a data frame containg a large set of breast cancer patients. For each tumor, we have about 30 different chracteristics and a marker signifying whether the tumor is cancerous or benign (0 is cancerous and 1 is benign)

Run the following code to upload the data set. We will save the data frame as `Cancer_feat`. Note the markers are saved automatically as `Cancerdata.target`

In [ ]:
Cancerdata=datasets.load_breast_cancer()
Cancer_feat=pd.DataFrame(Cancerdata.data, columns=Cancerdata.feature_names)

Cancer_feat

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


**a.** As you can see each tumor is described by 30 values. We want to reduce this to 2 values without loosing too much information. Before we can do that we need to be able to somehow compare values from different columns. As you can see, right now value differ a lot (for example, look at the columns `mean area` and `mean smoothness`). So, we want to "normalize" it somehow. We will use z-score. So now, do the following:

Convert data frame to NumPy array, then *standardize* every column. To do that, find mean and standard deviation of every column (NumPy has commands for that). Then subtract mean from column and divide by standard deviation. ( i.e  $\frac{x-\mu}{\sigma}$). Save it as a new array $M$. This is standardized matrix.

[Hint: If you have an 2-dim array $A$ with $n$ columns and a vector ${\bf v}$ with $n$ entries then $A-{\bf v}$ will subtract each entry of ${\bf v}$ from all entries of a corresponding column in $A$. Same applies to all other operations.]

**b.** Find covariance matrix $C=\frac{M^TM}{n-1}$, where $n$ is amount of rows in $M$. Then find eigenvalues and eigenvectors of $C$. Save the eigevector corresponding to the largest eigenvalue as $v1$ and the eigenvector corresponding to the second largest eigenvalue as $v2$. Make sure, that norms of $v1$ and $v2$ are 1.

Covariance matrix shows you how much your data is spread. Values in the main diagonal of $C$ are just variances of each column of $M$ (see if that makes sense by trying to see the calculation of the first value $c_{1,1}$). Then values of other entries are covariances of corresponding columns of $M$ (this means how much relation is there between those two columns). Knowing this, what do you think $v1$ represents?

**Answer:**

**c.** Project all the data points (rows) of $M$ onto a line spanned by $v1$ saving results as $X$ and onto a line spanned by $v2$ saving those as $Y$.
(Note: since norms of $v1$ and $v2$ are 1, this is nothing more than matrix vector multiplication.

Finally, plot the scatter plot using $X$ as x-coordinates and $Y$ as y-coordinates. Since we want to differentiate between cancerous and benign tumors, let's color points based on that. Before ploting the scatter plot use this `color=np.array(Cancerdata.target)` and use `c=color` or `hue=color` inside scatter plot depending on what you are using to plot scatter plot.

**d.** Do you think the $X$ and $Y$ values we got differentiate well between cancerous and benign tumors? Elaborate.

**Answer:**